In [69]:
import os
import pickle
from datetime import datetime, timedelta
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd


os.chdir("./bda-midterm-project/data")
retval = os.getcwd()
print("当前工作目录为 %s" % retval + "\n")

files = os.listdir(retval)
print("View all datasets:")
for f in files:
    print(f)

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: './bda-midterm-project/data'

## 前處理(日期)

In [2]:
def load_df(filepath, preview=True):
    print(f"\n----- Loading {filepath}... -----")
    df = pd.read_csv(filepath)
    print(f"Size of dataframe: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    if preview:
        print(df.head())
    return df

In [3]:
news = load_df("news_filtered.csv", preview=False)
mobile01 = load_df("mobile01_filtered.csv", preview=False)
dcard = load_df("dcard_filtered.csv", preview=False)
ptt = load_df("ptt_filtered.csv", preview=False)


----- Loading news_filtered.csv... -----
Size of dataframe: (8617, 9)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url']

----- Loading mobile01_filtered.csv... -----
Size of dataframe: (683, 10)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url', 'content_type']

----- Loading dcard_filtered.csv... -----
Size of dataframe: (6665, 10)
Columns: ['id', 'forum', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url', 'content_type']

----- Loading ptt_filtered.csv... -----
Size of dataframe: (2013, 9)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url']


In [70]:
transaction = load_df("個股交易數據-2年_filtered.csv", preview=True)


----- Loading 個股交易數據-2年_filtered.csv... -----
Size of dataframe: (11614, 8)
Columns: ['stock_name', 'stock_symbol', 'open', 'high', 'low', 'close', 'volume', 'date']
  stock_name  stock_symbol  open  high   low  close  volume  \
0       長榮航太          2645  91.9  94.7  89.5   89.5    8434   
1       長榮航太          2645  90.3  91.5  88.5   89.5    4004   
2       長榮航太          2645  88.6  88.7  84.1   84.7    3228   
3       長榮航太          2645  85.4  90.2  85.4   89.9    3584   
4       長榮航太          2645  89.1  91.4  88.1   90.8    2509   

                  date  
0  2023-03-14 00:00:00  
1  2023-03-15 00:00:00  
2  2023-03-16 00:00:00  
3  2023-03-17 00:00:00  
4  2023-03-20 00:00:00  


In [71]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11614 entries, 0 to 11613
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stock_name    11614 non-null  object 
 1   stock_symbol  11614 non-null  int64  
 2   open          11614 non-null  float64
 3   high          11614 non-null  float64
 4   low           11614 non-null  float64
 5   close         11614 non-null  float64
 6   volume        11614 non-null  int64  
 7   date          11614 non-null  object 
dtypes: float64(4), int64(2), object(2)
memory usage: 726.0+ KB


In [72]:
transaction['date'] = pd.to_datetime(transaction['date'])
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11614 entries, 0 to 11613
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   stock_name    11614 non-null  object        
 1   stock_symbol  11614 non-null  int64         
 2   open          11614 non-null  float64       
 3   high          11614 non-null  float64       
 4   low           11614 non-null  float64       
 5   close         11614 non-null  float64       
 6   volume        11614 non-null  int64         
 7   date          11614 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 726.0+ KB


In [73]:
transaction['date'] = pd.to_datetime(transaction['date'].dt.date)
transaction.head(3)

,stock_name,stock_symbol,open,high,low,close,volume,date
0,長榮航太,2645,91.9,94.7,89.5,89.5,8434,2023-03-14
1,長榮航太,2645,90.3,91.5,88.5,89.5,4004,2023-03-15
2,長榮航太,2645,88.6,88.7,84.1,84.7,3228,2023-03-16


In [75]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11614 entries, 0 to 11613
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   stock_name    11614 non-null  object        
 1   stock_symbol  11614 non-null  int64         
 2   open          11614 non-null  float64       
 3   high          11614 non-null  float64       
 4   low           11614 non-null  float64       
 5   close         11614 non-null  float64       
 6   volume        11614 non-null  int64         
 7   date          11614 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 726.0+ KB


In [74]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8617 entries, 0 to 8616
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           8617 non-null   object        
 1   p_type       8617 non-null   object        
 2   s_name       8617 non-null   object        
 3   s_area_name  8617 non-null   object        
 4   post_time    8617 non-null   datetime64[ns]
 5   title        8617 non-null   object        
 6   author       4835 non-null   object        
 7   content      8617 non-null   object        
 8   page_url     8617 non-null   object        
 9   date         8617 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 673.3+ KB


In [78]:
#to datetime & keep date only
news['post_time'] = pd.to_datetime(news['post_time'])
news["date"] = pd.to_datetime(news['post_time'].dt.date)
news.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,date
0,1646085283937_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 04:10:00,致富膠囊－新興美元主權債的三大投資主題,NaN,與一年前相比，新興市場債券在2022年初面臨更加悲觀的預期。但我們認為新興債市仍有上漲空間，...,https://tw.stock.yahoo.com/news/%E8%87%B4%E5%A...,2022-03-01
1,1646101482056_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 10:10:37,《航運股》中菲行拓展全球版圖 合資成立鳳凰城子公司,NaN,【時報記者莊丙農台北報導】中菲行(5609)持續拓展全球版圖，與BC Logistics公司...,https://tw.stock.yahoo.com/news/%E8%88%AA%E9%8...,2022-03-01
2,1646105982338_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 11:24:17,《台北股市》賣機票買船票！外資上周賣超華航最多、買超長榮第一,NaN,【時報-台北電】外資上周在集中市場賣超1665.44億元，寫下台股史上罕見的最大周賣超，就個...,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...,2022-03-01


In [79]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8617 entries, 0 to 8616
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           8617 non-null   object        
 1   p_type       8617 non-null   object        
 2   s_name       8617 non-null   object        
 3   s_area_name  8617 non-null   object        
 4   post_time    8617 non-null   datetime64[ns]
 5   title        8617 non-null   object        
 6   author       4835 non-null   object        
 7   content      8617 non-null   object        
 8   page_url     8617 non-null   object        
 9   date         8617 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(8)
memory usage: 673.3+ KB


In [193]:
#forum data
mobile01['post_time'] = pd.to_datetime(mobile01['post_time'])
mobile01["date"] = pd.to_datetime(mobile01['post_time'].dt.date)

dcard['post_time'] = pd.to_datetime(dcard['post_time'])
dcard["date"] = pd.to_datetime(dcard['post_time'].dt.date)

ptt['post_time'] = pd.to_datetime(ptt['post_time'])
ptt["date"] = pd.to_datetime(ptt['post_time'].dt.date)

mobile01.info()
dcard.info()
ptt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            683 non-null    object        
 1   p_type        683 non-null    object        
 2   s_name        683 non-null    object        
 3   s_area_name   683 non-null    object        
 4   post_time     683 non-null    datetime64[ns]
 5   title         683 non-null    object        
 6   author        683 non-null    object        
 7   content       675 non-null    object        
 8   page_url      683 non-null    object        
 9   content_type  683 non-null    object        
 10  date          683 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 58.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        ---------

## 決定每一日總指數是漲、跌、還是持平

In [81]:
date_discuss = news['date'].unique() #所有討論日
date_open = transaction['date'].unique() #所有開市日

print(len(date_discuss),len(date_open))

682 485


In [82]:
#檢查公司列表
companylist =  transaction["stock_name"].unique()
companylist

array(['長榮航太', '台灣高鐵', '龍德造船', '亞航', '宅配通', '建新國際', '台船', '台驊投控', '漢翔',
       '慧洋-KY', '遠雄港', '志信', '華航', '裕民', '新興', '陽明', '台航', '中櫃', '嘉里大榮',
       '四維航', '萬海', '長榮航', '榮運', '長榮', '中航'], dtype=object)

In [162]:
#每日交易量表
totalvolume_day = transaction.groupby("date")["volume"].sum()

totalvolume_day = pd.pivot_table(
                                    data = transaction,
                                    values = "volume",
                                    index = "date",
                                    aggfunc = "sum"
                                ).reset_index()
totalvolume_day

,date,volume
0,2022-03-01,1033290
1,2022-03-02,982387
2,2022-03-03,1173551
3,2022-03-04,1530623
4,2022-03-07,1368171
...,...,...
480,2024-02-20,182543
481,2024-02-21,272585
482,2024-02-22,280407
483,2024-02-23,205490


In [194]:
transaction.tail(5)

,stock_name,stock_symbol,open,high,low,close,volume,date
11609,萬海,2615,50.50,50.6,49.45,49.65,6870,2024-02-23
11610,遠雄港,5607,62.20,62.8,61.60,61.90,2152,2024-02-23
11611,志信,2611,17.65,17.7,17.30,17.30,1551,2024-02-23
11612,陽明,2609,49.20,49.4,48.60,48.70,24867,2024-02-26
11613,建新國際,8367,43.95,44.0,43.65,43.90,158,2024-02-26


In [86]:
#按公司、日期排序
sorted_trans = transaction.sort_values(by=["stock_name", "date"]).reset_index(drop=True)
sorted_trans.head(20)

,stock_name,stock_symbol,open,high,low,close,volume,date
0,中櫃,2613,32.05,32.25,30.60,31.90,4754,2022-03-01
1,中櫃,2613,32.45,35.05,31.80,35.05,5695,2022-03-02
2,中櫃,2613,36.55,38.55,36.10,38.55,22998,2022-03-03
3,中櫃,2613,39.55,41.40,39.05,39.40,28020,2022-03-04
4,中櫃,2613,39.35,40.55,38.60,39.55,12794,2022-03-07
5,中櫃,2613,40.05,40.95,35.70,35.80,8148,2022-03-08
6,中櫃,2613,38.00,39.35,37.50,39.35,2899,2022-03-09
7,中櫃,2613,43.25,43.25,40.45,42.00,22029,2022-03-10
8,中櫃,2613,40.50,42.75,39.40,39.50,11690,2022-03-11
9,中櫃,2613,40.45,41.20,38.75,38.75,6000,2022-03-14


In [87]:
name = "initial name"
for index, row in sorted_trans.iterrows():
    
    #每支股票頭一天漲幅設為0
    if row["stock_name"] != name:
        change = 0
        sorted_trans.loc[index,"change"] = change
        
    #其餘計算漲幅
    else:
        today_close = row["close"]                              #print("today close ",today_close)
        lastday_close = sorted_trans.loc[index-1]["close"]      #print("last day close ",lastday_close)
        change = (today_close-lastday_close) / lastday_close    #print(change)
        sorted_trans.loc[index,"change"] = change

    name = row["stock_name"]

    #當日該股票交易量佔總交易量比例
    today_totalvolume = totalvolume_day.loc[totalvolume_day["date"] == row["date"],"volume"].iloc[0]    #print(row["date"], " totalvolume = ", today_totalvolume)
    sorted_trans.loc[index,"partition"] = row["volume"] / today_totalvolume

    #加權分數 (漲幅*佔比)
    sorted_trans.loc[index,"score"] = sorted_trans.loc[index,"change"] * sorted_trans.loc[index,"partition"]

sorted_trans.head(10)

,stock_name,stock_symbol,open,high,low,close,volume,date,change,partition,score
0,中櫃,2613,32.05,32.25,30.60,31.90,4754,2022-03-01,0.000000,0.004601,0.000000
1,中櫃,2613,32.45,35.05,31.80,35.05,5695,2022-03-02,0.098746,0.005797,0.000572
2,中櫃,2613,36.55,38.55,36.10,38.55,22998,2022-03-03,0.099857,0.019597,0.001957
3,中櫃,2613,39.55,41.40,39.05,39.40,28020,2022-03-04,0.022049,0.018306,0.000404
4,中櫃,2613,39.35,40.55,38.60,39.55,12794,2022-03-07,0.003807,0.009351,0.000036
5,中櫃,2613,40.05,40.95,35.70,35.80,8148,2022-03-08,-0.094817,0.005458,-0.000518
6,中櫃,2613,38.00,39.35,37.50,39.35,2899,2022-03-09,0.099162,0.002636,0.000261
7,中櫃,2613,43.25,43.25,40.45,42.00,22029,2022-03-10,0.067344,0.014741,0.000993
8,中櫃,2613,40.50,42.75,39.40,39.50,11690,2022-03-11,-0.059524,0.013876,-0.000826
9,中櫃,2613,40.45,41.20,38.75,38.75,6000,2022-03-14,-0.018987,0.006955,-0.000132


In [88]:
sorted_trans.tail(10)

,stock_name,stock_symbol,open,high,low,close,volume,date,change,partition,score
11604,龍德造船,6753,82.6,83.8,82.4,82.7,164,2024-02-02,0.001211,0.001068,0.000001
11605,龍德造船,6753,82.7,82.7,81.3,82.4,289,2024-02-05,-0.003628,0.002118,-0.000008
11606,龍德造船,6753,81.2,82.1,81.2,81.7,221,2024-02-15,-0.008495,0.001002,-0.000009
11607,龍德造船,6753,81.3,82.0,81.1,81.2,342,2024-02-16,-0.006120,0.002650,-0.000016
11608,龍德造船,6753,81.2,82.1,80.2,80.3,507,2024-02-19,-0.011084,0.004291,-0.000048
11609,龍德造船,6753,80.4,80.6,79.4,79.5,521,2024-02-20,-0.009963,0.002854,-0.000028
11610,龍德造船,6753,79.5,80.7,79.2,80.3,239,2024-02-21,0.010063,0.000877,0.000009
11611,龍德造船,6753,80.5,81.0,79.8,79.9,271,2024-02-22,-0.004981,0.000966,-0.000005
11612,龍德造船,6753,80.0,80.0,78.7,78.8,553,2024-02-23,-0.013767,0.002691,-0.000037
11613,龍德造船,6753,78.8,80.8,78.8,80.6,370,2024-02-26,0.022843,0.002436,0.000056


In [89]:
#以其中一天來看
sorted_trans[sorted_trans["date"] == sorted_trans["date"][1]]

,stock_name,stock_symbol,open,high,low,close,volume,date,change,partition,score
1,中櫃,2613,32.45,35.05,31.80,35.05,5695,2022-03-02,0.098746,0.005797,5.724413e-04
486,中航,2612,53.70,54.70,53.20,53.20,1712,2022-03-02,-0.014815,0.001743,-2.581769e-05
971,亞航,2630,19.25,19.25,18.90,18.90,207,2022-03-02,-0.018182,0.000211,-3.831114e-06
1456,台灣高鐵,2633,29.50,29.55,29.30,29.55,4342,2022-03-02,0.001695,0.004420,7.491266e-06
1941,台航,2617,37.00,38.50,36.35,37.05,15340,2022-03-02,0.001351,0.015615,2.110139e-05
2426,台船,2208,21.50,21.75,21.40,21.70,1553,2022-03-02,0.004630,0.001581,7.318719e-06
2911,台驊投控,2636,135.00,137.50,134.00,136.50,2912,2022-03-02,-0.003650,0.002964,-1.081828e-05
3396,嘉里大榮,2608,43.85,43.85,43.40,43.50,443,2022-03-02,0.001151,0.000451,5.189211e-07
3881,四維航,5608,44.50,45.85,44.25,44.50,15731,2022-03-02,-0.006696,0.016013,-1.072302e-04
4366,宅配通,2642,75.70,76.00,75.00,75.00,426,2022-03-02,-0.013158,0.000434,-5.705759e-06


In [182]:
sorted_trans[sorted_trans["date"] == sorted_trans["date"][1]]["partition"].sum()

1.0

In [183]:
#每日類股漲跌總平均 (按日期把score加總)
date_score = pd.pivot_table(
                                    data = sorted_trans,
                                    values = "score",
                                    index = "date",
                                    aggfunc = "sum"
                                ).reset_index()

date_score.tail(10)

,date,score
475,2024-02-02,-0.024534
476,2024-02-05,0.001354
477,2024-02-15,-0.013068
478,2024-02-16,0.007366
479,2024-02-19,0.011047
480,2024-02-20,0.019335
481,2024-02-21,0.013905
482,2024-02-22,0.016655
483,2024-02-23,-0.010850
484,2024-02-26,0.009710


In [184]:
date_score["score"].describe()

count    485.000000
mean       0.001692
std        0.026176
min       -0.180639
25%       -0.011188
50%        0.001617
75%        0.015470
max        0.174840
Name: score, dtype: float64

In [185]:
#根據上面敘述統計，以總漲幅/跌幅0.01為區分，中間的50%資料為持平，不會納入機器學習(-1)
date_score["rise_fall"] = np.where(date_score['score'] >= 0.01, 1, np.where(date_score['score'] <= -0.01, 0, -1))

In [186]:
date_score.tail(10)

,date,score,rise_fall
475,2024-02-02,-0.024534,0
476,2024-02-05,0.001354,-1
477,2024-02-15,-0.013068,0
478,2024-02-16,0.007366,-1
479,2024-02-19,0.011047,1
480,2024-02-20,0.019335,1
481,2024-02-21,0.013905,1
482,2024-02-22,0.016655,1
483,2024-02-23,-0.010850,0
484,2024-02-26,0.009710,-1


In [187]:
date_score.isna().any()

date         False
score        False
rise_fall    False
dtype: bool

## 資料貼標

In [108]:
#開市日
open_dates =date_score["date"].unique()
print(type(open_dates))
print(open_dates)

<class 'pandas.core.arrays.datetimes.DatetimeArray'>
<DatetimeArray>
['2022-03-01 00:00:00', '2022-03-02 00:00:00', '2022-03-03 00:00:00',
 '2022-03-04 00:00:00', '2022-03-07 00:00:00', '2022-03-08 00:00:00',
 '2022-03-09 00:00:00', '2022-03-10 00:00:00', '2022-03-11 00:00:00',
 '2022-03-14 00:00:00',
 ...
 '2024-02-02 00:00:00', '2024-02-05 00:00:00', '2024-02-15 00:00:00',
 '2024-02-16 00:00:00', '2024-02-19 00:00:00', '2024-02-20 00:00:00',
 '2024-02-21 00:00:00', '2024-02-22 00:00:00', '2024-02-23 00:00:00',
 '2024-02-26 00:00:00']
Length: 485, dtype: datetime64[ns]


In [195]:
#共五種label，設為討論發生後1-5天
day_option = [1,2,3,4,5]

#將4個dataset 放入 list
data_list = []
data_list.append(news)
data_list.append(mobile01)
data_list.append(dcard)
data_list.append(ptt)

for data in data_list:
    for day_after in day_option:
        new_col = "label_day" + str(day_after)

        for index, row in data.iterrows():
            
            #date_to_find為討論日+day_after
            date_to_find = pd.to_datetime(row["date"]) + pd.Timedelta(days= day_after) #print(index,"today",row["date"], "find", date_to_find)
            
            #當日有開市
            if date_to_find in open_dates:
                data.loc[index,new_col] = date_score.loc[date_score["date"] == date_to_find,"rise_fall"].iloc[0] #print("found,", data.loc[index,new_col])
            
            #當日無開市->延後到下一個開市日
            #最後面的一些討論日期比市場數據晚，無法驗證故標為-1
            else:
                next_date = date_score[date_score['date'] > date_to_find]['date'].min()
                if not pd.isnull(next_date):
                    data.loc[index,new_col] = date_score.loc[date_score["date"] == next_date, "rise_fall"].iloc[0] #print("not found, use ", next_date, data.loc[index,new_col])
                else:
                    data.loc[index,new_col] = -1 #print("out of range, label as -1")


news.head(10)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,date,label_day1,label_day2,label_day3,label_day4,label_day5
0,1646085283937_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 04:10:00,致富膠囊－新興美元主權債的三大投資主題,NaN,與一年前相比，新興市場債券在2022年初面臨更加悲觀的預期。但我們認為新興債市仍有上漲空間，...,https://tw.stock.yahoo.com/news/%E8%87%B4%E5%A...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
1,1646101482056_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 10:10:37,《航運股》中菲行拓展全球版圖 合資成立鳳凰城子公司,NaN,【時報記者莊丙農台北報導】中菲行(5609)持續拓展全球版圖，與BC Logistics公司...,https://tw.stock.yahoo.com/news/%E8%88%AA%E9%8...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
2,1646105982338_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 11:24:17,《台北股市》賣機票買船票！外資上周賣超華航最多、買超長榮第一,NaN,【時報-台北電】外資上周在集中市場賣超1665.44億元，寫下台股史上罕見的最大周賣超，就個...,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
3,1646112285252_1_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 13:02:37,憂俄烏戰爭令勞動力緊缺＋日貨櫃航運股創高，貨櫃三雄盤中股價漲逾4%,NaN,【財訊快報／記者劉居全報導】隨著俄烏戰爭爆發，外媒表示，根據國際航運商會（ICS）發表的聲明...,https://tw.stock.yahoo.com/news/%E6%86%82%E4%B...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
4,1646115880343_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 14:04:52,長榮集團爭奪戰提早開始！航空歸弟弟、哥哥力保航運？兄弟「戰和」第一觀察指標是這個,NaN,長榮國際經營權變天，將持續引爆長榮(2603)經營權戰火！上週2/25長榮國際舉行股東臨時會...,https://tw.stock.yahoo.com/news/%E9%95%B7%E6%A...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
5,1646116802322_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 14:28:47,《航運股》中菲行拓展北美版圖 再添鳳凰城新據點,NaN,【時報-台北電】中菲行(5609)持續拓展全球版圖，完成投資其於美國亞利桑那州鳳凰城的合資子...,https://tw.stock.yahoo.com/news/%E8%88%AA%E9%8...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
6,1646123075992_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 15:44:00,鋼鐵/航運領漲 台股漲246點收17898點/收復5日線,NaN,MoneyDJ新聞 2022-03-01 15:44:49 記者 新聞中心 報導美股周一漲跌...,https://tw.stock.yahoo.com/news/%E9%8B%BC%E9%9...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
7,1646124890072_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 16:36:52,永豐期貨：鋼鐵、航運股衝鋒，台股強彈逾兩百點,NaN,【財訊快報／編輯部】台指期週二上漲302點至17912點。價差方面，台指期轉為正價差13.7...,https://tw.stock.yahoo.com/news/%E6%B0%B8%E8%B...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
8,1646126689392_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-01 17:16:55,2 月權證交易降溫 前十大標的華航擠下台積電,NaN,2 月交易天數減少，台股也呈現震盪整理，權證市場交易略顯降溫，證交所統計，2 月權證成交值為...,https://tw.stock.yahoo.com/news/2-%E6%9C%88%E6...,2022-03-01,-1.0,1.0,-1.0,0.0,0.0
9,1646170816408_N01,news,Yahoo股市(收錄2022/12/1前資料),財經新聞,2022-03-02 04:10:00,新興市場教父墨比爾斯：規避地緣政治風險 可買黃金多元化投資,NaN,新興市場教父墨比爾斯（Mark Mobius）指出，比特幣價格近期飆漲，是俄羅斯被歐美嚴厲制...,https://tw.stock.yahoo.com/news/%E6%96%B0%E8%8...,2022-03-02,1.0,-1.0,0.0,0.0,0.0


In [196]:
news.to_csv('news_filtered_labeled.csv')
mobile01.to_csv('mobile01_filtered_labeled.csv')
dcard.to_csv('dcard_filtered_labeled.csv')
ptt.to_csv('ptt_filtered_labeled.csv')

In [177]:
#原本使用dictionary存
# labels = {}

# for i in range(1,len(day_option)+1):
#     key = f"{i} day"
#     labels[key] = news[f"label_day{i}"].to_list()